In [1]:
# ============================================================================
# ⚙️ 기본 라이브러리 및 시스템 유틸리티
# ============================================================================
import os
import gc
import warnings
import time
import json
import joblib
import traceback
from dotenv import load_dotenv
from collections import Counter
from numba import jit # JIT 컴파일러
from datetime import datetime, timedelta # 날짜/시간 처리

# ============================================================================
# 📊 데이터 처리 및 분석
# ============================================================================
import numpy as np
import pandas as pd
import pandas_ta as ta # 기술적 분석 라이브러리

# ============================================================================
# 📈 머신러닝 (Scikit-learn)
# ============================================================================

# 📌 전처리 및 선택
from sklearn.feature_selection import (
    SelectKBest, RFE,
    mutual_info_classif, mutual_info_regression
)
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler, StandardScaler,MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight # 클래스 불균형 처리

# 📌 모델 (기본)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPClassifier # Multi-layer Perceptron

# 📌 모델 (앙상블)
from sklearn.ensemble import (
    AdaBoostClassifier, AdaBoostRegressor,
    BaggingClassifier, BaggingRegressor,
    ExtraTreesClassifier, ExtraTreesRegressor,
    GradientBoostingClassifier, GradientBoostingRegressor,
    HistGradientBoostingClassifier, # 히스토그램 기반 GB
    RandomForestClassifier, RandomForestRegressor,
    StackingClassifier, StackingRegressor,
    VotingClassifier, VotingRegressor
)

# 📌 평가 지표
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, # 분류
    mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error, # 회귀
    classification_report
)

# ============================================================================
# 🚀 부스팅 계열 모델
# ============================================================================
import optuna # 하이퍼파라미터 최적화

from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from lightgbm.callback import early_stopping
from xgboost import XGBClassifier, XGBRegressor

# ============================================================================
# 🧠 딥러닝 (TensorFlow/Keras)
# ============================================================================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    # 기본
    Input, Dense, Flatten, Dropout, Activation,
    # RNN/시퀀스
    LSTM, GRU, SimpleRNN, Bidirectional, TimeDistributed, RepeatVector,
    # CNN
    Conv1D, MaxPooling1D, AveragePooling1D,
    GlobalAveragePooling1D, GlobalMaxPooling1D,
    # 정규화
    BatchNormalization, LayerNormalization,
    # 유틸리티
    Concatenate, Add, Multiply, Lambda, Reshape, Permute
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2, l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ============================================================================
# ⏱️ 시계열 분석 (Statsmodels)
# ============================================================================
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.var_model import VAR

# ============================================================================
# ⚡ PyTorch (선택적)
# ============================================================================
try:
    import torch
    import torch.nn as nn
except ImportError:
    # PyTorch가 설치되지 않은 경우 무시
    pass



2025-11-22 10:08:09.839415: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-22 10:08:09.839462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-22 10:08:09.840854: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-22 10:08:09.847795: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-22 10:08:10.625654: W tensorflow/comp

In [2]:
# ==========================================================================
# 1. 유틸리티 함수 (지표 추가용)
# ==========================================================================

def add_indicator_to_df(df_ta, indicator):
    """pandas_ta 지표 결과를 DataFrame에 안전하게 추가"""
    if indicator is None:
        return

    if isinstance(indicator, pd.DataFrame) and not indicator.empty:
        for col in indicator.columns:
            df_ta[col] = indicator[col]
    elif isinstance(indicator, pd.Series) and not indicator.empty:
        colname = indicator.name if indicator.name else 'Unnamed'
        df_ta[colname] = indicator

def safe_add(df_ta, func, *args, **kwargs):
    """지표 생성 시 오류 방지를 위한 래퍼 함수"""
    try:
        result = func(*args, **kwargs)
        add_indicator_to_df(df_ta, result)
        return True
    except Exception as e:
        return False

# ==========================================================================
# 2. 기술적 지표 생성 (Sniper Optimization)
# ==========================================================================

def calculate_technical_indicators(df):
    """
    [1억 현물 투자용 최적화 지표]
    - 중복 지표(TEMA, DEMA 등) 제거
    - Swing High/Low Breakout(돌파 매매) 지표 추가
    """
    df = df.sort_values('date').reset_index(drop=True)
    df_ta = df.copy()

    close = df['ETH_Close']
    high = df.get('ETH_High', close)
    low = df.get('ETH_Low', close)
    volume = df.get('ETH_Volume', pd.Series(index=df.index, data=1))
    open_ = df.get('ETH_Open', close)

    # 1. [변동성] ATR 및 볼린저 밴드
    df_ta['ATR_14'] = ta.atr(high, low, close, length=14)
    bb = ta.bbands(close, length=20, std=2)
    if bb is not None:
        df_ta['BB_WIDTH'] = bb.iloc[:, 2]  # Bandwidth

    # 2. [추세] 신뢰도 높은 이평선만 유지
    df_ta['SMA_20'] = ta.sma(close, length=20)
    df_ta['SMA_50'] = ta.sma(close, length=50)
    df_ta['EMA_12'] = ta.ema(close, length=12)
    
    # 추세 점수 (정배열 여부)
    df_ta['TREND_SCORE'] = (close > df_ta['SMA_20']).astype(int) + (df_ta['SMA_20'] > df_ta['SMA_50']).astype(int)

    # 3. [모멘텀] RSI, MACD
    df_ta['RSI_14'] = ta.rsi(close, length=14)
    safe_add(df_ta, ta.macd, close, fast=12, slow=26, signal=9)

    # 4. [거래량] OBV, MFI, 상대 거래량
    df_ta['OBV'] = ta.obv(close, volume)
    df_ta['MFI_14'] = ta.mfi(high, low, close, volume, length=14)
    df_ta['VOLUME_RATIO'] = volume / (volume.rolling(20).mean() + 1e-8)

    # 5. [패턴] 윗꼬리/아랫꼬리
    df_ta['UPPER_SHADOW'] = (high - np.maximum(close, open_)) / (high - low + 1e-9)
    df_ta['LOWER_SHADOW'] = (np.minimum(close, open_) - low) / (high - low + 1e-9)

    # 6. [핵심] Swing Breakout (저항/지지 돌파)
    for window in [5, 20, 60]:
        swing_high = high.rolling(window).max().shift(1)
        swing_low = low.rolling(window).min().shift(1)
        
        # 위치 (1.0 돌파 시 매수 신호)
        df_ta[f'PRICE_VS_HIGH_{window}d'] = close / (swing_high + 1e-9)
        df_ta[f'PRICE_VS_LOW_{window}d'] = close / (swing_low + 1e-9)
        
        # 돌파 강도 (ATR 대비)
        df_ta[f'BREAKOUT_STR_{window}d'] = (close - swing_high) / (df_ta['ATR_14'] + 1e-9)

    return df_ta

# ==========================================================================
# 3. 파생 변수 및 전처리 함수 
# ==========================================================================

def add_enhanced_cross_crypto_features(df):
    df_enhanced = df.copy()
    df_enhanced['eth_return'] = df['ETH_Close'].pct_change().fillna(0)
    
    if 'BTC_Close' in df.columns:
        df_enhanced['btc_return'] = df['BTC_Close'].pct_change().fillna(0)
        # 상관관계
        for window in [7, 30]:
            df_enhanced[f'eth_btc_corr_{window}d'] = (
                df_enhanced['eth_return'].rolling(window).corr(df_enhanced['btc_return'])
            ).fillna(0)
        # 스프레드
        df_enhanced['eth_btc_spread'] = df_enhanced['eth_return'] - df_enhanced['btc_return']
        
    return df_enhanced

def add_price_lag_features_first(df):
    """가격 패턴 학습용 Lag"""
    df_new = df.copy()
    close = df['ETH_Close']
    # 핵심적인 1, 2, 3, 5, 10일 전만 확인
    for lag in [1, 2, 3, 5, 10]:
        df_new[f'return_lag{lag}'] = close.pct_change(periods=lag).shift(1)
    return df_new

def add_interaction_features(df):
    df_interact = df.copy()
    # 거래량 실린 지표 확인
    if 'RSI_14' in df.columns and 'VOLUME_RATIO' in df.columns:
        df_interact['RSI_Volume_Strength'] = df['RSI_14'] * df['VOLUME_RATIO']
    return df_interact

def add_volatility_regime_features(df):
    df_regime = df.copy()
    # 변동성 체제 확인
    if 'ATR_14' in df.columns:
        atr_ma = df['ATR_14'].rolling(20).mean()
        df_regime['high_volatility_regime'] = (df['ATR_14'] > atr_ma).astype(int)
    return df_regime

def add_percentile_features(df):
    df_pct = df.copy()
    # 현재 가격 위치 (랭크)
    if 'ETH_Close' in df.columns:
        df_pct['price_rank_250d'] = df['ETH_Close'].rolling(250).rank(pct=True)
    return df_pct

def preprocess_non_stationary_features(df):
    """외부 데이터(TVL 등) 정규화"""
    df_proc = df.copy()
    
    prefixes_to_transform = [
        'eth_', 'aave_', 'lido_', 'makerdao_', 'uniswap_', 'curve_', 'chain_',
        'l2_', 'sp500_', 'gold_', 'dxy_', 'vix_', 'usdt_'
    ]
    
    exclude_prefixes = ['fg_', 'funding_']
    exclude_keywords = [
        '_pct_', '_ratio', '_lag', '_volatility', '_corr', '_beta', '_spread',
        'eth_return', 'btc_return', 'eth_log_return',
        'RSI', 'MFI', 'CCI', 'ADX', 'BOP', 'AROON', 'PRICE_VS', 'BREAKOUT'
    ]
    
    cols_to_transform = []
    for col in df_proc.columns:
        col_lower = col.lower()
        if col.startswith(tuple(prefixes_to_transform)):
            if not col.startswith(tuple(exclude_prefixes)):
                if not any(k.lower() in col_lower for k in exclude_keywords):
                    cols_to_transform.append(col)
                    
    cols_to_drop = []
    for col in cols_to_transform:
        series = df_proc[col].fillna(method='ffill').replace(0, 1e-9)
        # 1일 변화율
        df_proc[f'{col}_pct_1d'] = series.pct_change(1)
        # 30일 이동평균 대비 비율
        ma_30 = series.rolling(window=30, min_periods=10).mean()
        df_proc[f'{col}_ma30_ratio'] = series / (ma_30 + 1e-9)
        cols_to_drop.append(col)

    df_proc = df_proc.drop(columns=cols_to_drop, errors='ignore')
    df_proc = df_proc.replace([np.inf, -np.inf], 0).fillna(0)
    
    return df_proc

# ==========================================================================
# 4. 결측치 및 초기 데이터 처리 (Burn-in)
# ==========================================================================

def handle_missing_values_paper_based(df_clean, train_start_date, is_train=True, train_stats=None):
    """초기 200일 데이터 제거 (안정성 확보)"""
    burn_in_period = 200
    if len(df_clean) > burn_in_period:
        df_clean = df_clean.sort_values('date').reset_index(drop=True)
        df_clean = df_clean.iloc[burn_in_period:].reset_index(drop=True)

    if isinstance(train_start_date, str):
        train_start_date = pd.to_datetime(train_start_date)
    df_clean = df_clean[df_clean['date'] >= train_start_date].reset_index(drop=True)
    
    target_cols = ['next_log_return', 'next_direction', 'next_close','next_open', 
                   'take_profit_price', 'stop_loss_price', 'exit_reason']
    feature_cols = [col for col in df_clean.columns if col not in target_cols + ['date']]
    
    df_clean[feature_cols] = df_clean[feature_cols].fillna(method='ffill').fillna(0)
    df_clean[feature_cols] = df_clean[feature_cols].replace([np.inf, -np.inf], 0)
    
    if is_train:
        return df_clean, {}
    else:
        return df_clean


# ==========================================================================
# 5. 데이터 누수 방지 및 정제
# ==========================================================================

def remove_raw_prices_and_transform(df, target_type, method):
    df_transformed = df.copy()
    
    if 'eth_log_return' not in df_transformed.columns:
        df_transformed['eth_log_return'] = np.log(df['ETH_Close'] / df['ETH_Close'].shift(1))
    
    remove_patterns = ['_Close', '_Open', '_High', '_Low', '_Volume']
    keep_keywords = [
        '_lag', '_position', '_ratio', '_range', '_change', '_corr', '_volatility', '_obv',
        'PRICE_VS', 'BREAKOUT', 'UPPER_SHADOW', 'LOWER_SHADOW', 'BB_WIDTH'
    ]
    
    cols_to_remove = [
        col for col in df_transformed.columns
        if any(p in col for p in remove_patterns)
        and not any(d in col.lower() for d in [k.lower() for k in keep_keywords])
    ]
    df_transformed.drop(cols_to_remove, axis=1, inplace=True)

    return_cols = [col for col in df_transformed.columns if 'return' in col.lower() and 'next' not in col]
    if return_cols:
        df_transformed[return_cols] = df_transformed[return_cols].fillna(0)

    return df_transformed


In [3]:
# ==========================================================================
# 6. 타겟 생성 
# ==========================================================================
from numba import jit

@jit(nopython=True)
def compute_targets_real_entry_numba(
    prices_close, prices_open, prices_high, prices_low, atr,
    lookahead, risk_reward, stop_mult
):
    n = len(prices_close)
    targets = np.zeros(n, dtype=np.int32) 
    
    for i in range(n - lookahead - 1):
        current_atr = atr[i]
        if np.isnan(current_atr) or current_atr <= 0:
            continue
            
        entry_price = prices_open[i + 1]
        
        stop_dist = current_atr * stop_mult
        profit_dist = stop_dist * risk_reward
        
        upper = entry_price + profit_dist
        lower = entry_price - stop_dist
        
        for j in range(1, lookahead + 1):
            idx = i + j
            
            future_high = prices_high[idx]
            future_low = prices_low[idx]

            if future_low <= lower:
                targets[i] = -1 
                break
            
            if future_high >= upper:
                targets[i] = 1 
                break
                
    return targets

def create_targets(df, lookahead=5, profit_mult=2.0, stop_mult=1.0, **kwargs):
    df_target = df.copy()
    
    if 'ATR_14' not in df_target.columns:
        df_target['ATR_14'] = ta.atr(
            df_target['ETH_High'], df_target['ETH_Low'], df_target['ETH_Close'], length=14
        )
    
    close = df_target['ETH_Close'].to_numpy()
    open_p = df_target['ETH_Open'].to_numpy()
    high = df_target['ETH_High'].to_numpy()
    low = df_target['ETH_Low'].to_numpy()
    atr = df_target['ATR_14'].fillna(method='ffill').fillna(0).to_numpy()
    
    rr_ratio = profit_mult / max(stop_mult, 0.1)
    
    targets = compute_targets_real_entry_numba(
        close, open_p, high, low, atr, lookahead, rr_ratio, stop_mult
    )
    
    df_target['next_direction'] = targets+1
    
    df_target['next_close'] = df_target['ETH_Close'].shift(-1) 
    df_target['next_open'] = df_target['ETH_Open'].shift(-1)   
    
    df_target['next_log_return'] = np.log(
        df_target['next_close'] / (df_target['next_open'] + 1e-9)
    )
    
    next_open = df_target['ETH_Open'].shift(-1)
    df_target['real_entry_price'] = next_open
    df_target['take_profit_price'] = next_open + (df_target['ATR_14'] * stop_mult * rr_ratio)
    df_target['stop_loss_price'] = next_open - (df_target['ATR_14'] * stop_mult)
    
    df_target.iloc[-lookahead:, df_target.columns.get_loc('next_direction')] = np.nan
    
    return df_target



In [4]:
def select_features_verified(X_train, y_train, task='class', top_n=20, verbose=True):
    if task == 'class':
        mi_scores = mutual_info_classif(X_train, y_train, random_state=42, n_neighbors=3)
    else:
        mi_scores = mutual_info_regression(X_train, y_train, random_state=42, n_neighbors=3)
    mi_idx = np.argsort(mi_scores)[::-1][:top_n]
    mi_features = X_train.columns[mi_idx].tolist()
    
    estimator = LGBMClassifier(n_estimators=100, random_state=42, verbose=-1) if task == 'class' else LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
    rfe = RFE(estimator=estimator, n_features_to_select=top_n, step=0.1, verbose=0)
    rfe.fit(X_train, y_train)
    rfe_features = X_train.columns[rfe.support_].tolist()

    rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1) if task == 'class' else RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    rf_idx = np.argsort(rf_model.feature_importances_)[::-1][:top_n]
    rf_features = X_train.columns[rf_idx].tolist()
    
    all_features = mi_features + rfe_features + rf_features
    feature_votes = Counter(all_features)
    selected_features = [feat for feat, _ in feature_votes.most_common(top_n)]
    
    if len(selected_features) < top_n:
        remaining = top_n - len(selected_features)
        for feat in mi_features:
            if feat not in selected_features:
                selected_features.append(feat)
                remaining -= 1
                if remaining == 0: break
    
    return selected_features, {}

def select_features_multi_target(X_train, y_train, target_type='direction', top_n=20):
    atr_col_name = 'ATR_14'
    if target_type == 'direction':
        selected, stats = select_features_verified(X_train, y_train['next_direction'], task='class', top_n=top_n)
        
        if atr_col_name not in selected and atr_col_name in X_train.columns:
            if len(selected) > 0: selected.pop()
            selected.insert(0, atr_col_name)
            
    print(f"\n[Feature Selection] Top {len(selected)} Features Selected:")
    print(f" -> {', '.join(selected)}")
    return selected, stats

def process_single_split(split_data, target_type='direction', top_n=20, fold_idx=None, trend_params=None): 
    
    train_df = split_data['train'] 
    val_df = split_data['val'] 
    test_df = split_data['test'] 
    fold_type = split_data.get('fold_type', 'unknown')

    print(f"\n{'='*80}")
    print(f" Processing Fold {fold_idx} ({fold_type})")
    print(f"{'='*80}")
    print(f" Train Period: {train_df['date'].min().date()} ~ {train_df['date'].max().date()} (N={len(train_df)})")
    print(f" Val   Period: {val_df['date'].min().date()} ~ {val_df['date'].max().date()} (N={len(val_df)})")
    print(f" Test  Period: {test_df['date'].min().date()} ~ {test_df['date'].max().date()} (N={len(test_df)})")

    train_processed, missing_stats = handle_missing_values_paper_based(train_df, train_df['date'].min(), is_train=True)
    val_processed = handle_missing_values_paper_based(val_df, val_df['date'].min(), is_train=False, train_stats=missing_stats)
    test_processed = handle_missing_values_paper_based(test_df, test_df['date'].min(), is_train=False, train_stats=missing_stats)

    target_cols = [
        'next_direction', 'next_log_return', 'next_close', 'next_open', 
        'take_profit_price', 'stop_loss_price', 
        'ATR_14', 'real_entry_price' 
    ]

    train_processed = train_processed.dropna(subset=target_cols).reset_index(drop=True)
    val_processed = val_processed.dropna(subset=target_cols).reset_index(drop=True)
    test_processed = test_processed.dropna(subset=target_cols).reset_index(drop=True)

    # [수정 4] ATR_14는 피처로도 써야 하므로 exclude_cols에서 제외
    exclude_cols = [col for col in target_cols if col != 'ATR_14'] + ['date']
    feature_cols = [col for col in train_processed.columns if col not in exclude_cols]
    
    X_train = train_processed[feature_cols]
    y_train = train_processed[target_cols]

    X_val = val_processed[feature_cols]
    y_val = val_processed[target_cols]

    X_test = test_processed[feature_cols]
    y_test = test_processed[target_cols]

    balance = y_train['next_direction'].value_counts(normalize=True).to_dict()
    print(f"[Class Balance] Train Set: {balance}")

    selected_features, selection_stats = select_features_multi_target(
        X_train, y_train, target_type=target_type, top_n=top_n
    )

    X_train_sel = X_train[selected_features]
    X_val_sel = X_val[selected_features]
    X_test_sel = X_test[selected_features]

    # ==================================================================
    # [수정 1] MinMaxScaler (-1 ~ 1) 적용
    # ==================================================================
    scaler = MinMaxScaler(feature_range=(-1, 1))

    X_train_scaled = scaler.fit_transform(X_train_sel)
    X_val_scaled = scaler.transform(X_val_sel)
    X_test_scaled = scaler.transform(X_test_sel)

    # ==================================================================
    # [수정 2] Numpy 배열을 DataFrame으로 복구 (LGBM 경고 해결)
    # ==================================================================
    X_train_final = pd.DataFrame(X_train_scaled, columns=selected_features)
    X_val_final = pd.DataFrame(X_val_scaled, columns=selected_features)
    X_test_final = pd.DataFrame(X_test_scaled, columns=selected_features)

    # ML용 Raw 데이터 (필요시 사용, 여기선 선택사항)
    X_train_raw = X_train_sel 

    if trend_params is None:
        trend_params = {}

    result = {
        'train': {
            # [수정 3] 모든 키가 MinMax Scaled 데이터를 가리키게 함
            'X_robust': X_train_final,   
            'X_standard': X_train_final, 
            'X_minmax': X_train_final,
            'X_raw': X_train_raw,
            'y': y_train.reset_index(drop=True), 
            'dates': train_processed['date'].reset_index(drop=True) 
        },
        'val': {
            'X_robust': X_val_final,
            'X_standard': X_val_final,
            'X_minmax': X_val_final,
            'X_raw': X_val_sel,
            'y': y_val.reset_index(drop=True), 
            'dates': val_processed['date'].reset_index(drop=True)
        },
        'test': {
            'X_robust': X_test_final,
            'X_standard': X_test_final,
            'X_minmax': X_test_final,
            'X_raw': X_test_sel,
            'y': y_test.reset_index(drop=True),
            'dates': test_processed['date'].reset_index(drop=True)
        },
        'scaler': scaler, 
        'stats': {
            'robust_scaler': scaler, 
            'standard_scaler': scaler,
            'minmax_scaler': scaler,
            'selected_features': selected_features,
            'selection_stats': selection_stats,
            'missing_stats': missing_stats,  
            'target_type': target_type,
            'target_cols': target_cols,
            'fold_type': fold_type,
            'fold_idx': fold_idx,
            'trend_window': trend_params.get('trend_window', 120),
            'trend_analysis_points': trend_params.get('trend_analysis_points', 5)
        }
    }

    return result


def split_walk_forward_method( df, train_start_date, final_test_start='2025-01-01', initial_train_size=800, val_size=150, test_size=150, step=150, gap_size=7 ):

    df_period = df[df['date'] >= train_start_date].copy()
    df_period = df_period.sort_values('date').reset_index(drop=True)

    if isinstance(final_test_start, str):
        final_test_start = pd.to_datetime(final_test_start)

    final_test_df = df_period[df_period['date'] >= final_test_start].copy()

    total_days = len(df_period)

    actual_step = test_size + gap_size
    min_required_days = initial_train_size + val_size + (gap_size * 2) + test_size

    print(f"\n{'='*80}")
    print(f"Reverse Rolling Walk-Forward ")
    print(f"{'='*80}")
    print(f"Total: {len(df_period)} days")
    print(f"Rolling train size: {initial_train_size} days ")
    print(f"Val: {val_size} days | Test: {test_size} days")
    print(f"Gap: {gap_size} days ")
    print(f"{'='*80}\n")

    folds = []
    current_test_end = total_days

    while True:
        test_end_idx = current_test_end
        test_start_idx = test_end_idx - test_size

        val_end_idx = test_start_idx - gap_size
        val_start_idx = val_end_idx - val_size

        train_end_idx = val_start_idx - gap_size
        train_start_idx = train_end_idx - initial_train_size

        if train_start_idx < 0:
            break

        train_fold = df_period.iloc[train_start_idx:train_end_idx].copy()
        val_fold = df_period.iloc[val_start_idx:val_end_idx].copy()
        test_fold = df_period.iloc[test_start_idx:test_end_idx].copy()

        folds.append({
            'train': train_fold,
            'val': val_fold,
            'test': test_fold,
            'fold_type': 'walk_forward_rolling_reverse'
        })

        current_test_end = test_start_idx - gap_size

    folds.reverse()

    for idx, fold in enumerate(folds):
        fold['fold_idx'] = idx + 1
        print(f"Fold {fold['fold_idx']} (walk_forward_rolling)")
        print(f"  Train: {len(fold['train']):4d}d  {fold['train']['date'].min().date()} ~ {fold['train']['date'].max().date()}")
        print(f"  Val:   {len(fold['val']):4d}d  {fold['val']['date'].min().date()} ~ {fold['val']['date'].max().date()}")
        print(f"  Test:  {len(fold['test']):4d}d  {fold['test']['date'].min().date()} ~ {fold['test']['date'].max().date()}\n")

    if len(final_test_df) > 0:
        pre_final_df = df_period[df_period['date'] < final_test_start].copy()

        final_val_end_idx = len(pre_final_df)
        final_val_start_idx = final_val_end_idx - val_size
        final_train_end_idx = final_val_start_idx - gap_size
        final_train_start_idx = final_train_end_idx - initial_train_size

        if final_train_start_idx < 0:
            final_train_start_idx = 0

        final_train_data = pre_final_df.iloc[final_train_start_idx:final_train_end_idx].copy()
        final_val_data = pre_final_df.iloc[final_val_start_idx:final_val_end_idx].copy()

        print(f"Fold {len(folds) + 1} (final_holdout)")
        print(f"  Train: {len(final_train_data):4d}d  {final_train_data['date'].min().date()} ~ {final_train_data['date'].max().date()}")
        print(f"  Val:   {len(final_val_data):4d}d  {final_val_data['date'].min().date()} ~ {final_val_data['date'].max().date()}")
        print(f"  Test:  {len(final_test_df):4d}d  {final_test_df['date'].min().date()} ~ {final_test_df['date'].max().date()}\n")

        folds.append({
            'train': final_train_data,
            'val': final_val_data,
            'test': final_test_df,
            'fold_idx': len(folds) + 1,
            'fold_type': 'final_holdout'
        })

    print(f"{'='*80}")
    print(f"Created {len(folds)} folds total")
    print(f"{'='*80}\n")

    return folds


def build_complete_pipeline_corrected(df_raw, train_start_date, **kwargs): 
    
    
    print(f"\n Pipeline Started... (Train Start: {train_start_date})")

    df = df_raw.copy()
    lookahead = kwargs.get('lookahead_candles', kwargs.get('lookahead', 5))
    profit_mult = kwargs.get('atr_multiplier_profit', 2.0)
    stop_mult = kwargs.get('atr_multiplier_stop', 1.0)

    df = add_price_lag_features_first(df)
    df = calculate_technical_indicators(df)
    df = add_enhanced_cross_crypto_features(df)
    df = add_volatility_regime_features(df)
    df = add_interaction_features(df)
    df = add_percentile_features(df)
    df = preprocess_non_stationary_features(df)

    df = create_targets(
        df, 
        lookahead=lookahead, 
        profit_mult=profit_mult, 
        stop_mult=stop_mult
    )

    df = remove_raw_prices_and_transform(df, 'direction', 'tvt')
    print(df.shape)
    splits = split_walk_forward_method(
        df, 
        train_start_date=train_start_date,
        final_test_start=kwargs.get('final_test_start', '2025-01-01'),
        initial_train_size=800,
        val_size=150,
        test_size=150,
        step=150,
        gap_size=lookahead
    )
    print(f" Data Split Completed. Total {len(splits)} folds generated.")

    result = []
    for fold in splits:
        res = process_single_split(
            fold, 
            top_n=kwargs.get('top_n', 20), 
            fold_idx=fold['fold_idx']
        )
        result.append(res)

    return result

In [5]:
import numpy as np
import pandas as pd
import optuna
import tensorflow as tf
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import joblib
import json

class DirectionModels:
    
    @staticmethod
    def get_class_weights(y_train):
        classes = np.unique(y_train)
        weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
        return dict(zip(classes, weights))

    @staticmethod
    def random_forest(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 300, 1000),
                'max_depth': trial.suggest_int('max_depth', 4, 12),
                'min_samples_split': trial.suggest_int('min_samples_split', 20, 150),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 60),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
                'ccp_alpha': trial.suggest_float('ccp_alpha', 1e-4, 1e-2, log=True),
                'class_weight': 'balanced',
                'random_state': 42,
                'n_jobs': -1
            }
            
            model = RandomForestClassifier(**params)
            model.fit(X_train, y_train)
            
            val_preds = model.predict(X_val)
            buy_precision = precision_score(y_val, val_preds, labels=[2], average='macro', zero_division=0)
            return buy_precision

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)
        
        best_model = RandomForestClassifier(**study.best_params, class_weight='balanced', random_state=42, n_jobs=-1)
        best_model.fit(X_train, y_train)
        return best_model

    @staticmethod
    def lightgbm(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            params = {
                'n_estimators': 1000,
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, 100),
                'max_depth': trial.suggest_int('max_depth', 4, 10),
                'min_child_samples': trial.suggest_int('min_child_samples', 30, 150),
                'subsample': trial.suggest_float('subsample', 0.5, 0.9),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-2, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 10.0, log=True),
                'objective': 'multiclass',
                'num_class': 3,
                'metric': 'multi_logloss',
                'class_weight': 'balanced',
                'verbosity': -1,
                'n_jobs': -1,
                'random_state': 42
            }
            
            model = LGBMClassifier(**params)
            callbacks = [lgb.early_stopping(stopping_rounds=30, verbose=False)]
            
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=callbacks)
            
            val_preds = model.predict(X_val)
            buy_precision = precision_score(y_val, val_preds, labels=[2], average='macro', zero_division=0)
            return buy_precision

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=25)
        
        best_params = study.best_params
        best_params.update({
            'n_estimators': 1000, 'objective': 'multiclass', 'num_class': 3,
            'metric': 'multi_logloss', 'class_weight': 'balanced', 'verbosity': -1, 
            'n_jobs': -1, 'random_state': 42
        })
        
        final_model = LGBMClassifier(**best_params)
        final_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)])
        return final_model

    @staticmethod
    def xgboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            params = {
                'n_estimators': 1000,
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'max_depth': trial.suggest_int('max_depth', 4, 10),
                'min_child_weight': trial.suggest_int('min_child_weight', 2, 15),
                'subsample': trial.suggest_float('subsample', 0.5, 0.9),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
                'gamma': trial.suggest_float('gamma', 0.5, 10.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-2, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 10.0, log=True),
                'objective': 'multi:softprob',
                'num_class': 3,
                'eval_metric': 'mlogloss',
                'tree_method': 'hist',
                'early_stopping_rounds': 30,
                'random_state': 42,
                'n_jobs': -1
            }
            
            model = XGBClassifier(**params)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
            
            val_preds = model.predict(X_val)
            buy_precision = precision_score(y_val, val_preds, labels=[2], average='macro', zero_division=0)
            return buy_precision

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=25)
        
        best_params = study.best_params
        best_params.update({
            'n_estimators': 1000, 'objective': 'multi:softprob', 'num_class': 3,
            'eval_metric': 'mlogloss', 'tree_method': 'hist', 'early_stopping_rounds': 50,
            'random_state': 42, 'n_jobs': -1
        })
        
        final_model = XGBClassifier(**best_params)
        final_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        return final_model

    @staticmethod
    def catboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            params = {
                'iterations': 1000,
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'depth': trial.suggest_int('depth', 4, 9),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 2, 15),
                'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
                'loss_function': 'MultiClass',
                'eval_metric': 'MultiClass',
                'auto_class_weights': 'Balanced',
                'logging_level': 'Silent',
                'random_seed': 42,
                'od_type': 'Iter',
                'od_wait': 30,
                'allow_writing_files': False
            }
            
            model = CatBoostClassifier(**params)
            model.fit(X_train, y_train, eval_set=(X_val, y_val))
            
            val_preds = model.predict(X_val)
            buy_precision = precision_score(y_val, val_preds, labels=[2], average='macro', zero_division=0)
            return buy_precision

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)
        
        best_params = study.best_params
        best_params.update({
            'iterations': 1000, 'loss_function': 'MultiClass', 'eval_metric': 'MultiClass',
            'auto_class_weights': 'Balanced', 'logging_level': 'Silent',
            'random_seed': 42, 'od_type': 'Iter', 'od_wait': 50, 'allow_writing_files': False
        })
        
        final_model = CatBoostClassifier(**best_params)
        final_model.fit(X_train, y_train, eval_set=(X_val, y_val))
        return final_model

    @staticmethod
    def lstm(X_train, y_train, X_val, y_val, input_shape):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        class_weights = DirectionModels.get_class_weights(y_train)

        def objective(trial):
            tf.keras.backend.clear_session()
            
            units = trial.suggest_int('units', 32, 128, step=16)
            dropout_rate = trial.suggest_float('dropout', 0.3, 0.6)
            l2_reg = trial.suggest_float('l2_reg', 1e-4, 1e-1, log=True)
            lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
            
            model = Sequential([
                Input(shape=input_shape),
                LSTM(units, return_sequences=False, 
                     kernel_regularizer=l1_l2(l2=l2_reg),
                     recurrent_regularizer=l1_l2(l2=l2_reg)),
                BatchNormalization(),
                Dropout(dropout_rate),
                Dense(units // 2, activation='relu', kernel_regularizer=l1_l2(l2=l2_reg)),
                Dropout(dropout_rate),
                Dense(3, activation='softmax')
            ])
            
            model.compile(
                optimizer=Adam(learning_rate=lr),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
            )
            
            es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
            
            model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=20,
                batch_size=64,
                class_weight=class_weights,
                callbacks=[es],
                verbose=0
            )
            
            val_probs = model.predict(X_val, verbose=0)
            val_preds = np.argmax(val_probs, axis=1)
            buy_precision = precision_score(y_val, val_preds, labels=[2], average='macro', zero_division=0)
            return buy_precision

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)
        
        best = study.best_params
        tf.keras.backend.clear_session()
        
        final_model = Sequential([
            Input(shape=input_shape),
            LSTM(best['units'], return_sequences=False,
                 kernel_regularizer=l1_l2(l2=best['l2_reg']),
                 recurrent_regularizer=l1_l2(l2=best['l2_reg'])),
            BatchNormalization(),
            Dropout(best['dropout']),
            Dense(best['units'] // 2, activation='relu', kernel_regularizer=l1_l2(l2=best['l2_reg'])),
            Dropout(best['dropout']),
            Dense(3, activation='softmax')
        ])
        
        final_model.compile(
            optimizer=Adam(learning_rate=best['lr']),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        final_es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
        final_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
        
        final_model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=50,
            batch_size=64,
            class_weight=class_weights,
            callbacks=[final_es, final_lr],
            verbose=0
        )
        return final_model
    
    
ML_MODELS_CLASSIFICATION = [
    {'index': 1, 'name': 'CatBoost', 'func': DirectionModels.catboost, 'needs_val': True},
    {'index': 2, 'name': 'RandomForest', 'func': DirectionModels.random_forest, 'needs_val': True},
    {'index': 3, 'name': 'LightGBM', 'func': DirectionModels.lightgbm, 'needs_val': True},
    {'index': 4, 'name': 'XGBoost', 'func': DirectionModels.xgboost, 'needs_val': True}
]

DL_MODELS_CLASSIFICATION = [
    {'index': 9, 'name': 'LSTM', 'func': DirectionModels.lstm, 'needs_val': True},
]

In [6]:
class ModelEvaluator:
    def __init__(self, save_models=False):
        self.results = []
        self.predictions = {}
        self.models = {} if save_models else None
        self.save_models = save_models
        self.best_thresholds = {}

    def optimize_threshold(self, y_true, adjusted_buy_prob, min_trades=5, reward_risk_ratio=2.0):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            th = trial.suggest_float('threshold', 0.4, 0.95)
            preds = np.where(adjusted_buy_prob >= th, 2, 1)
            n_trades = np.sum(preds == 2)
            
            if n_trades < min_trades:
                return -99999.0 
            
            real_wins = np.sum((preds == 2) & (y_true == 2))
            real_losses = n_trades - real_wins
            
            total_return = (real_wins * reward_risk_ratio) - (real_losses * 1.0)
            return total_return

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=30) 
        return study.best_params['threshold'], study.best_value

    def _calculate_metrics(self, y_true, adjusted_buy_prob, dataset_name, optimal_threshold=None, reward_risk_ratio=2.0):
        # y_true는 이제 확실하게 1차원 정수 배열(0,1,2)입니다.
        
        metrics = {}
        if optimal_threshold is not None:
            th_preds = np.where(adjusted_buy_prob >= optimal_threshold, 2, 1)
            n_trades = np.sum(th_preds == 2)
            
            if n_trades > 0:
                real_wins = np.sum((th_preds == 2) & (y_true == 2))
                real_losses = n_trades - real_wins
                
                win_rate = real_wins / n_trades
                total_return_r = (real_wins * reward_risk_ratio) - (real_losses * 1.0)
                
                gross_profit = real_wins * reward_risk_ratio
                gross_loss = real_losses * 1.0
                profit_factor = gross_profit / gross_loss if gross_loss > 0 else 99.0
                
                expectancy = total_return_r / n_trades
            else:
                win_rate = 0.0
                expectancy = 0.0
                total_return_r = 0.0
                profit_factor = 0.0
                
            metrics[f'{dataset_name}_Opt_Threshold'] = optimal_threshold
            metrics[f'{dataset_name}_Opt_Trades'] = n_trades
            metrics[f'{dataset_name}_Opt_WinRate'] = win_rate
            metrics[f'{dataset_name}_Opt_Expectancy'] = expectancy
            metrics[f'{dataset_name}_Opt_Total_Return_R'] = total_return_r
            metrics[f'{dataset_name}_Opt_Profit_Factor'] = profit_factor
            
        return metrics

    def _extract_target_label(self, y_data):
        """
        [수정됨] y_data에서 정답 라벨(next_direction)만 안전하게 추출하는 헬퍼 함수
        """
        if isinstance(y_data, pd.DataFrame):
            if 'next_direction' in y_data.columns:
                return y_data['next_direction'].values.astype(int)
            else:
                # 컬럼명이 없으면 첫 번째 컬럼을 타겟으로 가정
                return y_data.iloc[:, 0].values.astype(int)
        elif isinstance(y_data, pd.Series):
            return y_data.values.astype(int)
        elif isinstance(y_data, np.ndarray):
            # 차원이 2 이상이고 컬럼이 여러 개면 첫 번째 컬럼(0번 인덱스)이 타겟이라고 가정
            if y_data.ndim > 1 and y_data.shape[1] > 1:
                return y_data[:, 0].astype(int)
            return y_data.astype(int)
        return np.array(y_data).astype(int)

    def evaluate_classification_model(self, model, X_train, y_train, X_val, y_val, 
                                    X_test, y_test, test_dates, 
                                    test_prices, test_atr, 
                                    model_name,
                                    is_deep_learning=False, profit_mult=2.0, stop_mult=1.0):
        
        rr_ratio = profit_mult / stop_mult if stop_mult > 0 else 2.0

        # 예측 확률 계산
        if is_deep_learning:
            train_prob = model.predict(X_train, verbose=0)
            val_prob = model.predict(X_val, verbose=0)
            test_prob = model.predict(X_test, verbose=0)
        else:
            train_prob = model.predict_proba(X_train)
            val_prob = model.predict_proba(X_val)
            test_prob = model.predict_proba(X_test)
            
        def calculate_adjusted_prob(prob_matrix):
            prob_sell = prob_matrix[:, 0]
            prob_buy = prob_matrix[:, 2]
            return prob_buy / (prob_buy + prob_sell + 1e-9)

        train_adj_prob = calculate_adjusted_prob(train_prob)
        val_adj_prob = calculate_adjusted_prob(val_prob)
        test_adj_prob = calculate_adjusted_prob(test_prob)

        # [수정] 정답 데이터(Label)를 안전하게 추출
        y_train_fixed = self._extract_target_label(y_train)
        y_val_fixed = self._extract_target_label(y_val)
        y_test_fixed = self._extract_target_label(y_test)

        # Threshold 최적화
        best_th, best_val_return = self.optimize_threshold(
            y_val_fixed, val_adj_prob, 
            min_trades=5, 
            reward_risk_ratio=rr_ratio
        )
        self.best_thresholds[model_name] = best_th
        
        # 메트릭 계산
        train_metrics = self._calculate_metrics(y_train_fixed, train_adj_prob, 'Train', best_th, rr_ratio)
        val_metrics = self._calculate_metrics(y_val_fixed, val_adj_prob, 'Val', best_th, rr_ratio)
        test_metrics = self._calculate_metrics(y_test_fixed, test_adj_prob, 'Test', best_th, rr_ratio)
        
        result = {
            'Model': model_name,
            'Best_Threshold': best_th,
            'Risk_Reward_Ratio': rr_ratio
        }
        result.update(train_metrics)
        result.update(val_metrics)
        result.update(test_metrics)
        
        self.results.append(result)
        
        test_pred_opt = np.where(test_adj_prob >= best_th, 2, 1)
        test_pred_raw_argmax = np.argmax(test_prob, axis=1)
        
        # 가격 데이터 형상 맞추기
        if hasattr(test_prices, 'values'): test_prices = test_prices.values
        if hasattr(test_atr, 'values'): test_atr = test_atr.values
        test_prices = test_prices.reshape(-1)
        test_atr = test_atr.reshape(-1)

        tp_prices = test_prices + (test_atr * profit_mult)
        sl_prices = test_prices - (test_atr * stop_mult)
        
        print(f"[{model_name}]")
        print(f"  Optuna Threshold: {best_th:.4f} (Val Return: {best_val_return:.2f}R)")
        print(f"  Test Performance -> Win Rate: {result['Test_Opt_WinRate']:.2%} | Trades: {result['Test_Opt_Trades']} | PF: {result['Test_Opt_Profit_Factor']:.2f}")
        print(f"  Test Simulation  -> Expectancy: {result['Test_Opt_Expectancy']:.2f}R | Total Return: {result['Test_Opt_Total_Return_R']:.2f}R")
        
        self.predictions[model_name] = pd.DataFrame({
            'date': test_dates,
            'actual': y_test_fixed, 
            'entry_price': test_prices,
            'atr': test_atr,
            'tp_price': tp_prices,
            'sl_price': sl_prices,
            'pred_raw': test_pred_raw_argmax,  
            'pred_opt': test_pred_opt,    
            'prob_sell': test_prob[:, 0],
            'prob_hold': test_prob[:, 1],
            'prob_buy': test_prob[:, 2],
            'adj_prob_buy': test_adj_prob
        })
        
        if self.save_models and self.models is not None:
            self.models[model_name] = model
        
        return result

    # 나머지 메서드 동일
    def get_summary_dataframe(self):
        return pd.DataFrame(self.results)
    def get_predictions_dict(self):
        return self.predictions
    def get_models_dict(self):
        return self.models or {}
    
    
class ModelTrainer:
    def __init__(self, evaluator, lookback=30):
        self.evaluator = evaluator
        self.lookback = lookback
    
    def create_sequences(self, X, y_target, lookback):
        Xs, ys = [], []
        for i in range(lookback, len(X)):
            Xs.append(X[i-lookback:i])
            ys.append(y_target[i])
        return np.array(Xs), np.array(ys)
    
    def _prepare_target(self, y_data):
        if isinstance(y_data, pd.DataFrame):
            if 'next_direction' in y_data.columns:
                return y_data['next_direction'].values.astype(int)
            else:
                return y_data.iloc[:, 0].values.astype(int)
        elif isinstance(y_data, pd.Series):
            return y_data.values.astype(int)
        else:
            return np.array(y_data).astype(int)

    def train_all_models(self, X_train, y_train_input, X_val, y_val_input, X_test, y_test_input, 
                        test_dates, 
                        test_prices, test_atr, 
                        profit_mult, stop_mult, 
                        ml_models=None, dl_models=None):
        
        y_train_target = self._prepare_target(y_train_input)
        y_val_target = self._prepare_target(y_val_input)
        
        if ml_models:
            for config in ml_models:
                try:
                    if config['needs_val']:
                        model = config['func'](X_train, y_train_target, X_val, y_val_target)
                    else:
                        model = config['func'](X_train, y_train_target)
                        
                    self.evaluator.evaluate_classification_model(
                        model, X_train, y_train_target, X_val, y_val_target, X_test, y_test_input, 
                        test_dates, test_prices, test_atr, 
                        config['name'], 
                        is_deep_learning=False,
                        profit_mult=profit_mult, 
                        stop_mult=stop_mult      
                    )
                    del model
                    gc.collect()
                except Exception as e:
                    print(f"Model {config['name']} Failed: {str(e)}")
                    traceback.print_exc()

        if dl_models:
            X_train_s, y_train_s = self.create_sequences(X_train, y_train_target, self.lookback)
            X_val_s, y_val_s = self.create_sequences(X_val, y_val_target, self.lookback)
            
            y_test_target = self._prepare_target(y_test_input)
            X_test_s, y_test_s = self.create_sequences(X_test, y_test_target, self.lookback)
            
            if isinstance(y_test_input, pd.DataFrame):
                y_test_input_s = y_test_input.iloc[self.lookback:].reset_index(drop=True)
            else:
                y_test_input_s = y_test_input[self.lookback:]
                
            test_dates_s = test_dates[self.lookback:]
            test_prices_s = test_prices[self.lookback:]
            test_atr_s = test_atr[self.lookback:]
            
            input_shape = (X_train_s.shape[1], X_train_s.shape[2])
            
            for config in dl_models:
                try:
                    tf.keras.backend.clear_session()
                    model = config['func'](X_train_s, y_train_s, X_val_s, y_val_s, input_shape)
                    
                    self.evaluator.evaluate_classification_model(
                        model, X_train_s, y_train_s, X_val_s, y_val_s, X_test_s, y_test_input_s, 
                        test_dates_s, test_prices_s, test_atr_s, 
                        config['name'], 
                        is_deep_learning=True,
                        profit_mult=profit_mult, 
                        stop_mult=stop_mult      
                    )
                    del model
                    tf.keras.backend.clear_session()
                    gc.collect()
                except Exception as e:
                    print(f"DL Model {config['name']} Failed: {str(e)}")
                    traceback.print_exc()

def save_fold_results(fold_idx, fold_type, evaluator, trial_name, fold_data):
    base_dir = f"model_results/{datetime.now().strftime('%Y-%m-%d')}/{trial_name}/fold_{fold_idx}_{fold_type}"
    os.makedirs(base_dir, exist_ok=True)
    
    summary = evaluator.get_summary_dataframe()
    summary.to_csv(f"{base_dir}/fold_summary.csv", index=False)
    
    for name, df in evaluator.get_predictions_dict().items():
        df.to_csv(f"{base_dir}/predictions_{name}.csv", index=False)
        
    for name, model in evaluator.get_models_dict().items():
        try:
            if isinstance(model, tf.keras.Model):
                model.save(f"{base_dir}/model_{name}.h5")
            else:
                joblib.dump(model, f"{base_dir}/model_{name}.pkl")
        except Exception as e:
            print(f"[Warning] Failed to save model {name}: {e}")
            
    if 'scaler' in fold_data:
        joblib.dump(fold_data['scaler'], f"{base_dir}/scaler.pkl")
        
    meta_data = {
        'selected_features': fold_data['stats']['selected_features'],
        'target_cols': fold_data['stats']['target_cols'],
        'fold_idx': fold_idx,
        'fold_type': fold_type,
        'trial_name': trial_name,
        'best_thresholds': evaluator.best_thresholds
    }
    
    with open(f"{base_dir}/metadata.json", 'w') as f:
        json.dump(meta_data, f, indent=4)
            
    return summary

In [7]:

def run_optuna_optimization(df_merged, n_trials=30):
    TIMESTAMP = datetime.now().strftime("%Y-%m-%d")
    RESULT_DIR = f"model_results/{TIMESTAMP}"
    os.makedirs(RESULT_DIR, exist_ok=True)

    with open(f"{RESULT_DIR}/optuna_log.csv", "w") as f:
        f.write("trial,lookahead,profit_mult,stop_mult,score\n")

    def objective(trial):
        lookahead = trial.suggest_int('lookahead', 5, 15)
        p_mult = trial.suggest_float('profit_mult',1.5,2.0, step=0.1)
        s_mult = trial.suggest_float('stop_mult',1.5, 2.0, step=0.1)
        
        trial_name = f"trial_{trial.number}_L{lookahead}_P{p_mult:.1f}_S{s_mult:.1f}"
        print(f"\n{'='*80}")
        print(f" Starting {trial_name}")
        
        try:
            pipeline_result = build_complete_pipeline_corrected(
                df_raw=df_merged,
                train_start_date='2020-01-01',
                final_test_start='2025-01-01',
                lookahead_candles=lookahead,
                atr_multiplier_profit=p_mult,
                atr_multiplier_stop=s_mult,
                top_n=20
            )
            
            fold_scores = []
            
            for i, fold_data in enumerate(pipeline_result):
                fold_idx = fold_data['stats']['fold_idx']
                fold_type = fold_data['stats']['fold_type']
                
                print(f"\n   >> Running Fold {fold_idx}/{len(pipeline_result)} ({fold_type})")
                
                evaluator = ModelEvaluator(save_models=True)
                trainer = ModelTrainer(evaluator, lookback=30)
                
                test_prices = fold_data['test']['y']['real_entry_price'].values
                test_atr = fold_data['test']['y']['ATR_14'].values
                
                trainer.train_all_models(
                    fold_data['train']['X_robust'],
                    fold_data['train']['y'],
                    fold_data['val']['X_robust'],
                    fold_data['val']['y'],   
                    fold_data['test']['X_robust'],
                    fold_data['test']['y'],  
                    fold_data['test']['dates'].values,
                    test_prices,
                    test_atr,
                    p_mult,
                    s_mult,
                    ml_models=ML_MODELS_CLASSIFICATION,
                    dl_models=DL_MODELS_CLASSIFICATION
                )
                
                summary = save_fold_results(fold_idx, fold_type, evaluator, trial_name, fold_data)
                
                if 'Test_Opt_Expectancy' in summary.columns:
                    score = summary['Test_Opt_Expectancy'].mean()
                else:
                    score = 0.0
                    
                fold_scores.append(score)
                
                del evaluator, trainer
                gc.collect()
            
            final_score = np.mean(fold_scores) if fold_scores else 0.0
            print(f"\n === Trial {trial.number} Finished. Final Score (Expectancy): {final_score:.4f}R ===")
            
            with open(f"{RESULT_DIR}/optuna_log.csv", "a") as f:
                f.write(f"{trial.number},{lookahead},{p_mult},{s_mult},{final_score}\n")
                
            return final_score
            
        except Exception as e:
            print(f" [Error] Trial {trial.number} Failed: {e}")
            traceback.print_exc()
            return 0.0

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)
    
    print("\n[Optuna] Optimization Completed!")
    print("Best Params:", study.best_params)
    return study

In [13]:
import os
import gc
import traceback
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    roc_auc_score, 
    classification_report
)
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# ============================================================================
# 기본 라이브러리 및 유틸리티
# ============================================================================


import gc
import time
import json
import joblib
import os
import warnings
from dotenv import load_dotenv
# 날짜/시간
from datetime import datetime, timedelta

# 데이터 처리
import numpy as np
import pandas as pd
import pandas_ta as ta
from collections import Counter
from numba import jit
# ============================================================================
# ML/DL 라이브러리 및 도구
# ============================================================================

# 하이퍼파라미터 최적화
import optuna

# Scikit-learn: 데이터 전처리
from sklearn.feature_selection import (
    SelectKBest, RFE,
    mutual_info_classif, mutual_info_regression
)
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler, StandardScaler

# Scikit-learn: 모델
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

# Scikit-learn: 앙상블 모델
from sklearn.ensemble import (
    AdaBoostClassifier, AdaBoostRegressor,
    BaggingClassifier, BaggingRegressor,
    ExtraTreesClassifier, ExtraTreesRegressor,
    GradientBoostingClassifier, GradientBoostingRegressor,
    HistGradientBoostingClassifier, 
    RandomForestClassifier, RandomForestRegressor,
    StackingClassifier, StackingRegressor,
    VotingClassifier, VotingRegressor
)

# Scikit-learn: 평가 지표
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, # 분류 지표
    mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error # 회귀 지표
)


from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from lightgbm.callback import early_stopping 
from xgboost import XGBClassifier, XGBRegressor

# TensorFlow/Keras 딥러닝
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    # 기본 레이어
    Input, Dense, Flatten, Dropout, Activation,
    # RNN 레이어
    LSTM, GRU, SimpleRNN, Bidirectional,
    # CNN 레이어
    Conv1D, MaxPooling1D, AveragePooling1D,
    GlobalAveragePooling1D, GlobalMaxPooling1D,
    # 정규화 레이어
    BatchNormalization, LayerNormalization,
    # Attention 레이어
    Attention, MultiHeadAttention,
    # 유틸리티 레이어
    Concatenate, Add, Multiply, Lambda,
    Reshape, Permute, RepeatVector, TimeDistributed
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 시계열 분석 (Statsmodels)
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.var_model import VAR

# PyTorch 
try:
    import torch
    import torch.nn as nn
except ImportError:
    pass




import optuna
from sklearn.ensemble import (
    RandomForestClassifier, AdaBoostClassifier, BaggingClassifier,
    GradientBoostingClassifier, ExtraTreesClassifier, StackingClassifier,
    VotingClassifier, HistGradientBoostingClassifier
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
df_merged=pd.read_csv("merge_data.csv")

df_merged.tail(1)

,date,BTC_Open,BTC_High,BTC_Low,BTC_Close,BTC_Volume,ETH_Open,ETH_High,ETH_Low,ETH_Close,...,chain_eth_chain_tvl,funding_fundingRate,l2_arbitrum_tvl,l2_optimism_tvl,l2_base_tvl,l2_zksync era_tvl,sp500_SP500,vix_VIX,gold_GOLD,dxy_DXY
2350,2025-11-20,136500000.0,138621000.0,129800000.0,129896000.0,4303.832306,4510000.0,4561000.0,4212000.0,4243000.0,...,143848669231,0.000072,2.777513e+09,290218197.0,4.308060e+09,40361104.0,6538.759766,26.42,4056.5,100.160004


In [9]:
# print(df_merged.shape)
# df_merged['date']=pd.to_datetime(df_merged['date'])
# lookahead=5
# p_mult=1.5
# s_mult=1.0

# pipeline_result = build_complete_pipeline_corrected(
#     df_raw=df_merged,
#     train_start_date='2020-01-01',
#     final_test_start='2025-01-01',
#     lookahead_candles=lookahead,
#     atr_multiplier_profit=p_mult,
#     atr_multiplier_stop=s_mult,
#     top_n=20
# )
# print(pipeline_result)

In [10]:
#display(first_fold['train'])

In [11]:
if __name__ == "__main__":
    if tf.config.list_physical_devices('GPU'):
        try:
            for gpu in tf.config.list_physical_devices('GPU'):
                tf.config.experimental.set_memory_growth(gpu, True)
            print(" GPU Detected & Memory Growth Set!")
        except RuntimeError as e:
            print(e)
    
    if 'df_merged' in locals() and not df_merged.empty:
        df_merged['date'] = pd.to_datetime(df_merged['date'])
        df_merged = df_merged.sort_values('date').reset_index(drop=True)

        print(f"Loaded Data: {len(df_merged)} rows ({df_merged['date'].min().date()} ~ {df_merged['date'].max().date()})")

        study = run_optuna_optimization(df_merged, n_trials=30)
        
        print("\n==================================================")
        print(f" Best Trial Value (Avg Buy Precision): {study.best_value:.4f}")
        print(f" Best Parameters: {study.best_params}")
        print("==================================================")
    else:
        print("[Error] 'df_merged' variable is not defined or empty. Please load data first.")


[I 2025-11-22 10:08:13,669] A new study created in memory with name: no-name-8c6d4f8e-8c19-41f9-91e7-21fa197a777e


 GPU Detected & Memory Growth Set!
Loaded Data: 2351 rows (2019-06-15 ~ 2025-11-20)

 Starting trial_0_L5_P1.5_S1.8

 Pipeline Started... (Train Start: 2020-01-01)


/tmp/ipykernel_1556960/1210223439.py:54: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ta['EMA_12'] = ta.ema(close, length=12)


(2351, 85)

Reverse Rolling Walk-Forward 
Total: 2151 days
Rolling train size: 800 days 
Val: 150 days | Test: 150 days
Gap: 5 days 

Fold 1 (walk_forward_rolling)
  Train:  800d  2020-04-21 ~ 2022-06-29
  Val:    150d  2022-07-05 ~ 2022-12-01
  Test:   150d  2022-12-07 ~ 2023-05-05

Fold 2 (walk_forward_rolling)
  Train:  800d  2020-09-23 ~ 2022-12-01
  Val:    150d  2022-12-07 ~ 2023-05-05
  Test:   150d  2023-05-11 ~ 2023-10-07

Fold 3 (walk_forward_rolling)
  Train:  800d  2021-02-25 ~ 2023-05-05
  Val:    150d  2023-05-11 ~ 2023-10-07
  Test:   150d  2023-10-13 ~ 2024-03-10

Fold 4 (walk_forward_rolling)
  Train:  800d  2021-07-30 ~ 2023-10-07
  Val:    150d  2023-10-13 ~ 2024-03-10
  Test:   150d  2024-03-16 ~ 2024-08-12

Fold 5 (walk_forward_rolling)
  Train:  800d  2022-01-01 ~ 2024-03-10
  Val:    150d  2024-03-16 ~ 2024-08-12
  Test:   150d  2024-08-18 ~ 2025-01-14

Fold 6 (walk_forward_rolling)
  Train:  800d  2022-06-05 ~ 2024-08-12
  Val:    150d  2024-08-18 ~ 2025-01-14
 


[Feature Selection] Top 20 Features Selected:
 -> SMA_50, SMA_20, eth_btc_corr_30d, usdt_totalBridgedToUSD_ma30_ratio, PRICE_VS_HIGH_20d, gold_GOLD_ma30_ratio, ATR_14, EMA_12, BB_WIDTH, OBV, MACD_12_26_9, PRICE_VS_LOW_60d, l2_base_tvl_ma30_ratio, MACDH_12_26_9, MFI_14, return_lag10, usdt_totalUnreleased_ma30_ratio, curve_curve-dex_eth_tvl_ma30_ratio, vix_VIX_ma30_ratio, dxy_DXY_ma30_ratio

   >> Running Fold 1/8 (walk_forward_rolling_reverse)
[CatBoost]
  Optuna Threshold: 0.4977 (Val Return: -60.00R)
  Test Performance -> Win Rate: 37.16% | Trades: 148 | PF: 0.49
  Test Simulation  -> Expectancy: -0.32R | Total Return: -47.17R
[RandomForest]
  Optuna Threshold: 0.7049 (Val Return: -1.33R)
  Test Performance -> Win Rate: 27.27% | Trades: 11 | PF: 0.31
  Test Simulation  -> Expectancy: -0.50R | Total Return: -5.50R
[LightGBM]
  Optuna Threshold: 0.5164 (Val Return: 0.17R)
  Test Performance -> Win Rate: 32.35% | Trades: 34 | PF: 0.40
  Test Simulation  -> Expectancy: -0.41R | Total Ret

...


   >> Running Fold 5/8 (walk_forward_rolling_reverse)
[CatBoost]
  Optuna Threshold: 0.5091 (Val Return: -17.17R)
  Test Performance -> Win Rate: 61.54% | Trades: 26 | PF: 1.33
  Test Simulation  -> Expectancy: 0.13R | Total Return: 3.33R
[RandomForest]
  Optuna Threshold: 0.6651 (Val Return: -0.00R)
  Test Performance -> Win Rate: 8.70% | Trades: 23 | PF: 0.08
  Test Simulation  -> Expectancy: -0.84R | Total Return: -19.33R
[LightGBM]
  Optuna Threshold: 0.5273 (Val Return: -12.00R)
  Test Performance -> Win Rate: 17.50% | Trades: 40 | PF: 0.18
  Test Simulation  -> Expectancy: -0.68R | Total Return: -27.17R
[XGBoost]
  Optuna Threshold: 0.4036 (Val Return: -109.67R)
  Test Performance -> Win Rate: 28.00% | Trades: 150 | PF: 0.32
  Test Simulation  -> Expectancy: -0.49R | Total Return: -73.00R


[W 2025-11-22 10:20:50,883] Trial 5 failed with parameters: {'units': 96, 'dropout': 0.5963305828536876, 'l2_reg': 0.0006843591194925379, 'lr': 0.001779530526219786} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_1556960/209885645.py", line 217, in objective
    model.fit(
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py", line 1807, in fit
    tmp_logs = self.train_function(iterator)
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/raid/invigoworks/

[W 2025-11-22 10:20:50,905] Trial 5 failed with value None.
[W 2025-11-22 10:20:50,906] Trial 0 failed with parameters: {'lookahead': 5, 'profit_mult': 1.5, 'stop_mult': 1.8} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_1556960/1343576181.py", line 43, in objective
    trainer.train_all_models(
  File "/tmp/ipykernel_1556960/2681119743.py", line 261, in train_all_models
    model = config['func'](X_train_s, y_train_s, X_val_s, y_val_s, input_shape)
  File "/tmp/ipykernel_1556960/209885645.py", line 233, in lstm
    study.optimize(objective, n_trials=20)
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/optuna/study/study.py", line 490, in optimize
    _optimize(
  File "/raid/invigoworks/anaconda3/lib/python3.10/site-packages/optuna/study/_optimize.py", line

[W 2025-11-22 10:20:50,908] Trial 0 failed with value None.


KeyboardInterrupt: 